
<h1  id="2" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:Red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Importing Libraries </h1>
           
 

<span style="font-family:cursive; font-size:15px;">
We are Importing Libraries. </span>


<span style="font-family:cursive; font-size:15px;">Libraries which need for
 </span>


<span style="font-family:cursive; font-size:15px;">Data visualization </span>


<span style="font-family:cursive; font-size:15px;">Making Model Architecture. </span>











In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import random
import zipfile
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:

import numpy as np
import sys
import tensorflow as tf
import os
import sys
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


%matplotlib inline
import matplotlib.image as img
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go
import itertools
import plotly.express as px

<h1  id="2.1" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px">  Setting Path and Extracting Files </h1>

In [ ]:


with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r') as train_zip:
    train_zip.extractall('.')  
image_dir = "../working/train/"
filenames = os.listdir(image_dir)
labels = [x.split(".")[0] for x in filenames]
data = pd.DataFrame({"filename": filenames, "label": labels})
data.head()

In [ ]:
IMAGE_FOLDER_PATH = "../working/train"
FILE_NAMES = os.listdir(IMAGE_FOLDER_PATH)
WIDTH = 150
HEIGHT = 150
# empty list
targets = list()
full_paths = list()
train_cats_dir = list()
train_dogs_dir = list()

# finding each file's target
for file_name in FILE_NAMES:
    target = file_name.split(".")[0] # target name
    full_path = os.path.join(IMAGE_FOLDER_PATH, file_name)
    
    if(target == "dog"):
        train_dogs_dir.append(full_path)
    if(target == "cat"):
        train_cats_dir.append(full_path)
    
    full_paths.append(full_path)
    targets.append(target)

dataset = pd.DataFrame() # make dataframe
dataset['image_path'] = full_paths # file path
dataset['target'] = targets # file's target
data.head()

<h1  id="3" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px">  Data distribuation of all data </h1>

In [ ]:
print(data['label'].value_counts())
counts = data['label'].value_counts()

Data is balanced and has only two categories.

In [ ]:
fig = go.Figure(go.Bar(
            x= counts.values,
            y=counts.index,
            orientation='h'))

fig.update_layout(title='Data Distribution in Bars',font_size=15,title_x=0.45)


fig.show()

fig=px.pie(counts.head(10),values= 'label', names=data['label'].unique(),hole=0.425)
fig.update_layout(title='Data Distribution of Data',font_size=15,title_x=0.45,annotations=[dict(text='Cat vs Dog',font_size=18, showarrow=False,height=800,width=700)])
fig.update_traces(textfont_size=15,textinfo='percent')
fig.show()

<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Displaying Images of Cat & Dog </h1>

In [ ]:

grouped_data = data.groupby("label")

num_images_per_category = 5

fig, axes = plt.subplots(len(grouped_data), num_images_per_category, figsize=(20, 20))

for i, (category, group) in enumerate(grouped_data):
  
    random_indices = random.sample(range(len(group)), num_images_per_category)

    for j, index in enumerate(random_indices):
        filename = group.iloc[index]["filename"]
        label = group.iloc[index]["label"]
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path)

        axes[i, j].imshow(image)
        axes[i, j].set_title("Label: " + label, fontsize = 30)
        

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img

def plot_images(directory, title):
    rows = 4
    cols = 4
    axes = []
    fig = plt.figure(figsize=(10, 10))
    i = 0

    for a in range(rows * cols):
        b = img.imread(directory[i])
        axes.append(fig.add_subplot(rows, cols, a + 1))
        plt.imshow(b)
        i += 1
    fig.tight_layout()
    plt.suptitle(title, fontsize=16)
    plt.show()

# Assuming you have directories train_cats_dir and train_dogs_dir

# For cats
plot_images(train_cats_dir, "Cat Images")

# For dogs
plot_images(train_dogs_dir, "Dog Images")


Images are of varying size.

Every time the cell is run different set of images will be displayed, one can scan the images of both categories. Presence of humans in some images could be a challenge for the model to classify. 

Some images have more than one cats or dogs respectively.



<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Find median size of image for model building </h1>

In [ ]:
 
image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]

image_width = []
image_height = []
for image_path in image_paths:
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    image_width.append(width)
    image_height.append(height)

median_width = np.median(image_width)
median_height = np.median(image_height)


print('median_size:', median_width,  'X', median_height)

Since CNN models can process square images thus setting the image size for processing as 
370 X 370. 



<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Create train, validation and test datasets </h1>

In [ ]:
labels = data['label']
X_train, X_temp = train_test_split(data, test_size=0.2, stratify=labels, random_state = 23)

label_test_val = X_temp['label']
X_test, X_val = train_test_split(X_temp, test_size=0.5, stratify=label_test_val, random_state = 23)

print ('X_train:', X_train['label'].value_counts())

print ('X_val:', X_val['label'].value_counts())

print ('X_test:', X_test['label'].value_counts())

All three splits of data has both categories in the same proportion (equally distributed) as that in the original data. 

<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Create image data generator
 </h1>

In [ ]:
batch_size = 64
size = (370, 370)
idg = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input)

In [ ]:
train_idg = idg.flow_from_dataframe(X_train, "train/", x_col= "filename", y_col= "label",
                                    batch_size = batch_size,
                                    target_size=size)

In [ ]:
val_idg = idg.flow_from_dataframe(X_val, "train/", x_col="filename", y_col="label",
                                  batch_size = batch_size,
                                  target_size = size, shuffle = False)

In [ ]:
test_idg = idg.flow_from_dataframe(X_test, "train/", x_col= "filename", y_col= "label",
                                    batch_size = batch_size,
                                    target_size=size, shuffle = False)

Set shuffle = False for validation and test dataset. By setting shuffle=False, the samples will be processed in the same order as they appear in the dataframe.

<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Download vgg16 model and freeze the weights
 </h1>

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=False, input_shape=(370, 370, 3))

In [ ]:
vgg16_model.summary()

In [ ]:
for layer in vgg16_model.layers:
  layer.trainable = False

By setting layer.trainable = False, freezing the weights of all the layers in the VGG16 model.The vgg16 derived weights will not be trained during the training of final model.


<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Define functional model
 </h1>

In [ ]:
flat = tf.keras.layers.Flatten() (vgg16_model.output)
dropout1 = tf.keras.layers.Dropout(0.2, name="Dropout1") (flat)
dense1 = tf.keras.layers.Dense(128, activation="relu") (dropout1)
dropout2 = tf.keras.layers.Dropout(0.2, name="Dropout2")(dense1)
output = tf.keras.layers.Dense(2, activation="softmax") (dropout2)

final_model = tf.keras.models.Model(inputs=[vgg16_model.input], outputs=[output])

<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Plot the final model
 </h1>

In [ ]:
tf.keras.utils.plot_model(final_model, show_shapes = True, show_layer_names=True)

In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(final_model, legend=True) 

<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px"> Compile the model
 </h1>

In [ ]:
final_model.compile(optimizer='adam',
                    loss=tf.keras.losses.categorical_crossentropy,
                    metrics = ["acc"])

 <h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px">Create callback
 </h1>

In [ ]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint("DogCat",
                                                monitor="val_loss",
                                                save_best_only=True)

The ModelCheckpoint callback will save the best model during training based on the validation loss. 

The saved model will be stored with the prefix "DogCat".

 <h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px">Train the model
 </h1>

In [ ]:
history = final_model.fit(train_idg, batch_size=batch_size, validation_data=val_idg, epochs = 8, callbacks=[model_ckpt])

 <h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px">Evaluate accuracy and loss for the model
 </h1>

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(8)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Validation loss start increasing after 3 epochs.


<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px">Check the accuracy of the model on test subset
 </h1>

In [ ]:
cat_dog_model = tf.keras.models.load_model("DogCat")

result = cat_dog_model.predict(test_idg)

result_argmax = np.argmax(result, axis=1)

y_true = test_idg.labels

y_pred = result_argmax

accuracy = (y_pred == y_true).mean()

print("Test Accuracy:", accuracy)

Vgg16 rocks! 98% accuracy  in our test subset that we split from training data.

In [ ]:
print(classification_report(y_true, y_pred))

Both the classes are classified well.



<h1  id="5" style="color:white;
           display:fill;
           border-radius:5px;
           background-color:red;
           font-size:350%;
           font-family:Verdana;
           letter-spacing:0.5px">Make predictions on kaggle test data for submission.
 </h1>

Unzip and extract test data

In [ ]:

with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip', 'r') as test1_zip:
    test1_zip.extractall('.')  
test_dir = "../working/test1/"
filenames = os.listdir(test_dir)
test_data = pd.DataFrame({"filename": filenames})
test_data['label'] = 'unknown'
test_data.head()

Create data genenerator for test data

In [ ]:

test1_idg =  idg.flow_from_dataframe(test_data, "test1/", x_col= "filename",y_col = 'label',
                                    batch_size = batch_size,
                                    target_size=size, shuffle = False)

Make predictions on test data

In [ ]:
test1_predict = cat_dog_model.predict(test1_idg)

test1_predict_argmax = np.argmax(test1_predict, axis=1)

y_test_pred = test1_predict_argmax

test_data['label'] = y_test_pred

test_data.head()



In [ ]:
train_idg.class_indices

In [ ]:
label_mapping = {0: 'cat', 1: 'dog'}
test_data['label'] = test_data['label'].map(label_mapping)
test_data.head()

Visualize the predicted results

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx in range(5):
    image_path = os.path.join(test_dir, test_data.iloc[idx]['filename'])
    image = Image.open(image_path)
    axes[idx].imshow(image)
    axes[idx].set_title("Label: " + test_data.iloc[idx]['label'])
    axes[idx].axis('off')
plt.show()


Made quite well predictions!

If you find the notebook helpful - Upvote